In [1]:
# This cell contains some utility functions to prepare and execute the benchmarks
import timeit
from statistics import median
from random import choice
from string import ascii_uppercase

def random_string(length):
    """Produce a random string made of *length* uppercase ascii characters"""
    return ''.join(choice(ascii_uppercase) for i in range(length))

def print_scaling(stmt, setup, sizes=[10000, 20000, 30000], repeat=False, units='us'):
    """Print scaling information for the statement *stmt*, executed after *setup*.
    
    The *setup* and *stmt* arguments take a template string where "{N}"
    will be replaced as the size of the input.
    
    The *repeat* flags determined if the setup needs to be run between
    each test run.
    """
    values = []
    for size in sizes:
        if repeat:
            timings = timeit.repeat(stmt.format(N=size),
                                    setup=setup.format(N=size),
                                    number=1, repeat=1000)
            values.append(min(timings))
        else:
            timings = timeit.repeat(stmt.format(N=size),
                                    setup=setup.format(N=size),
                                    number=1000, repeat=3)
            values.append(min(t/1000 for t in timings))
    unit_factor = {'us': 1e6,
                   'ms': 1e3}[units]
    
    print(' | '.join('N = {} t = {:.2f} ({})'.format(n, t * unit_factor, units) for n, t in zip(sizes, values)))

# Lists

In [2]:
print_scaling('collection.pop()',
              setup='collection = list(range({N}))')

N = 10000 t = 0.12 (us) | N = 20000 t = 0.11 (us) | N = 30000 t = 0.12 (us)


In [3]:
print_scaling('collection.pop(0)',
              setup='collection = list(range({N}))')

N = 10000 t = 3.57 (us) | N = 20000 t = 7.27 (us) | N = 30000 t = 11.86 (us)


In [4]:
print_scaling('collection.append(1)',
                        setup='collection = list(range({N}))')

N = 10000 t = 0.07 (us) | N = 20000 t = 0.07 (us) | N = 30000 t = 0.08 (us)


In [11]:
print_scaling('collection.insert(0, 1)',
              setup='collection = list(range({N}))')

N = 10000 t = 6.63 (us) | N = 20000 t = 14.96 (us) | N = 30000 t = 17.20 (us)


In [12]:
print_scaling('collection.insert(5000, 1)',
              setup='collection = list(range({N}))')

N = 10000 t = 5.34 (us) | N = 20000 t = 9.81 (us) | N = 30000 t = 14.80 (us)


In [13]:
setup_code = '''
import random
random.seed(42)

collection = list(range({N}))
'''
print_scaling('collection.index(random.randint(0, {N}))',
              setup=setup_code)

N = 10000 t = 90.16 (us) | N = 20000 t = 175.22 (us) | N = 30000 t = 261.10 (us)


In [32]:
from bisect import bisect_left

def index_bisect(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

setup_code = '''
from __main__ import index_bisect

import random
random.seed(42)

collection = list(range({N}))
'''
    
print_scaling('index_bisect(collection, random.randint(0, {N}))',
              setup=setup_code)

N = 10000 t = 2.89 (us) | N = 20000 t = 3.46 (us) | N = 30000 t = 4.18 (us)


# Deque

In [15]:
print_scaling('collection.pop()',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.09 (us) | N = 20000 t = 0.08 (us) | N = 30000 t = 0.08 (us)


In [16]:
print_scaling('collection.popleft()',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.11 (us) | N = 20000 t = 0.08 (us) | N = 30000 t = 0.09 (us)


In [17]:
print_scaling('collection.append(1)',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.20 (us) | N = 20000 t = 0.15 (us) | N = 30000 t = 0.16 (us)


In [18]:
print_scaling('collection.appendleft(1)',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.15 (us) | N = 20000 t = 0.16 (us) | N = 30000 t = 0.15 (us)


In [108]:
print_scaling('collection[0]',
              setup='from collections import deque; collection = deque(range({N}))')
print_scaling('collection[{N} - 1]',
              setup='from collections import deque; collection = deque(range({N}))')
print_scaling('collection[int({N}/2)]',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.37 | N = 20000 t = 0.41 | N = 30000 t = 0.45
N = 10000 t = 0.37 | N = 20000 t = 0.42 | N = 30000 t = 0.43
N = 10000 t = 1.14 | N = 20000 t = 1.71 | N = 30000 t = 2.48


# Dictionaries

In [171]:
setup_code = '''
from __main__ import random_string
collection = {{random_string(16) : i for i in range({N})}}
'''

print_scaling('"ITEM" in collection',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

print_scaling('collection["ITEM"] if "ITEM" in collection else None',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

print_scaling('collection["ITEM"] = 0',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

N = 1000 t = 0.04 (us) | N = 2000 t = 0.03 (us) | N = 3000 t = 0.03 (us)
N = 1000 t = 0.04 (us) | N = 2000 t = 0.04 (us) | N = 3000 t = 0.04 (us)
N = 1000 t = 0.06 (us) | N = 2000 t = 0.06 (us) | N = 3000 t = 0.07 (us)


In [19]:
setup_code = '''
from __main__ import random_string
collection = {{random_string(16) : i for i in range({N})}}
'''

print_scaling('"X"  * {N} in collection',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

N = 1000 t = 0.95 (us) | N = 2000 t = 1.45 (us) | N = 3000 t = 2.10 (us)


In [177]:
setup_code = '''
from uuid import uuid4
from collections import Counter
from __main__ import counter_defaultdict, counter_dict
import random

random.seed(42)
collection = [random.randint(0, 100) for i in range({N})]
'''
from collections import defaultdict
def counter_defaultdict(items):
    counter = defaultdict(int)
    for item in items:
        counter[item] += 1
    return counter

def counter_dict(items): 
    counter = {} 
    for item in items: 
        if item not in counter: 
            counter[item] = 0 
        else: 
            counter[item] += 1 
    return counter

print_scaling('Counter(collection)',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

print_scaling('counter_defaultdict(collection)',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

print_scaling('counter_dict(collection)',
              setup=setup_code,
              sizes=[1000, 2000, 3000])


N = 1000 t = 51.48 (us) | N = 2000 t = 96.73 (us) | N = 3000 t = 140.26 (us)
N = 1000 t = 120.90 (us) | N = 2000 t = 238.27 (us) | N = 3000 t = 359.60 (us)
N = 1000 t = 111.96 (us) | N = 2000 t = 197.13 (us) | N = 3000 t = 282.79 (us)


In [21]:
from string import ascii_uppercase
import random
random.seed(42)

strings = [random_string(32) for i in range(10000)]

In [22]:
%timeit next(s for s in strings if s.startswith('AA'))

10000 loops, best of 3: 32.5 µs per loop


In [23]:
from patricia import trie
string_trie = trie(**{s: 0 for s in strings})

In [24]:
%timeit next(string_trie.iter('AA'))

The slowest run took 9.05 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.6 µs per loop


In [25]:
setup_code = '''
from itertools import islice
from random import choice, seed, shuffle
from string import ascii_uppercase
seed(42)

# Initialize strings without letter 'A'
strings = [''.join(choice(ascii_uppercase.replace('A', '')) for i in range(32)) for j in range({N})]

# We make sure we only have 10 results that start with 'AA'
strings = ['AA' + s[2:] if i < 10 else s for i, s in enumerate(strings)]

# Randomization
shuffle(strings)
'''

print_scaling('[s for s in strings if s.startswith("AA")]',
              setup=setup_code,
              sizes=[10000, 20000, 30000],
              repeat=False, units='us')

N = 10000 t = 1954.30 (us) | N = 20000 t = 4106.75 (us) | N = 30000 t = 7102.93 (us)


In [26]:
setup_code = '''
from patricia import trie
from random import choice, seed, shuffle
from string import ascii_uppercase
from itertools import islice
seed(42)

# Initialize strings without letter 'A'
strings = [''.join(choice(ascii_uppercase.replace('A', '')) for i in range(32)) for j in range({N})]

# We make sure we only have 10 results that start with 'AA'
strings = ['AA' + s[2:] if i < 10 else s for i, s in enumerate(strings)]

# Randomization
shuffle(strings)
strings_dict = {{s: 0 for s in strings}}
strings_trie = trie(**strings_dict)
'''

print_scaling('list(strings_trie.iter("AA"))',
              setup=setup_code,
              sizes=[10000, 20000, 30000], units='us')

N = 10000 t = 17.74 (us) | N = 20000 t = 17.48 (us) | N = 30000 t = 17.96 (us)


# Heaps

In [27]:
# Creating a heap
import heapq
collection = [10, 3, 3, 4, 5, 6]
heapq.heapify(collection)

# Extracting and inserting elements
heapq.heappop(collection)
heapq.heappush(collection, 1)

In [31]:
# The PriorityQueue class
from queue import PriorityQueue
collection = [10, 3, 3, 4, 5, 6]

queue = PriorityQueue()
for element in collection:
    queue.put(element)

queue.get(element)

3

# Caching

In [36]:
def fibonacci(n):
    if n < 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

# Non-memoized version
%timeit fibonacci(20)

100 loops, best of 3: 6 ms per loop


In [40]:
import timeit
setup_code = '''
from functools import lru_cache
from __main__ import fibonacci
fibonacci_memoized = lru_cache(maxsize=None)(fibonacci)
'''

results = timeit.repeat('fibonacci_memoized(20)',
                        setup=setup_code,
                        repeat=1000,
                        number=1)
print("Fibonacci took {:.2f} us".format(min(results)))

Fibonacci took 0.01 us


# Comprehensions and generators

In [7]:
def loop(): 
    res = {} 
    for i in range(100000): 
        res[i] = i
    return res
 
def comprehension(): 
    return {i: i for i in range(100000)}

%timeit loop()
%timeit comprehension() 

100 loops, best of 3: 13.2 ms per loop
100 loops, best of 3: 12.8 ms per loop


In [5]:
def loop(): 
    res = [] 
    for i in range(100000): 
        res.append(i * i) 
    return sum(res) 
 
def comprehension(): 
    return sum([i * i for i in range(100000)]) 
 
def generator(): 
    return sum(i * i for i in range(100000)) 

%timeit loop() 
%timeit comprehension() 
%timeit generator() 

100 loops, best of 3: 16.1 ms per loop
100 loops, best of 3: 10.1 ms per loop
100 loops, best of 3: 12.4 ms per loop


In [76]:
%load_ext memory_profiler

In [83]:
import os

numbers = range(1000000)

def map_comprehension(numbers):
    a = [n * 2 for n in numbers]
    b = [n ** 2 for n in a]
    c = [n ** 0.33 for n in b]
    return max(c)

%memit map_comprehension(numbers) 

peak memory: 166.33 MiB, increment: 102.54 MiB


In [84]:
def map_normal(numbers):
    a = map(lambda n: n * 2, numbers)
    b = map(lambda n: n ** 2, a)
    c = map(lambda n: n ** 0.33, b)
    return max(c)

%memit map_normal(numbers)

peak memory: 71.04 MiB, increment: 0.00 MiB
